In [29]:
import yfinance as yf
import datetime as dt
import pandas_datareader as pdr
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

## Descarga datos y creación dataset

In [7]:
sample_per = 60
stock = 'EURGBP=X'

end = dt.date.today()- dt.timedelta(days=2)
start = end - dt.timedelta(days=(sample_per))
interval = '1h'

In [8]:
database = yf.download(stock, start=start, end=end, interval=interval)

[*********************100%***********************]  1 of 1 completed


In [9]:
# Creamos indicadores
database['200ma'] = database['Adj Close'].ewm(span=200).mean()
database['50ma'] = database['Adj Close'].ewm(span=50, adjust=False).mean()
database['20ma'] = database['Adj Close'].ewm(span=20, adjust=False).mean()


#Generamos una copia, añadimos la columna de diferencia y eliminamos valores NaN
data = database.copy()

data['Dif'] = data['Adj Close'].diff()

data.dropna(inplace=True)

In [10]:
# Creamos el indicador Up_Down segun si es positivo o negativo (sube o baja precio)
lista =[]
for index, row in data.iterrows():
    if row['Dif'] > 0:
        lista.append(1)
    else:
        lista.append(0)

serie = pd.Series(lista, index = data.index)

#Lo añadimos al DataFrame y nos quedamos con las columnas que nos interesan
data['Up_Down'] = serie

data = data[['Adj Close', '200ma', '50ma', '20ma','Up_Down']]

## Preparacion datos para el LSTM

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1047 entries, 2021-06-29 00:00:00+01:00 to 2021-08-27 22:00:00+01:00
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Adj Close  1047 non-null   float64
 1   200ma      1047 non-null   float64
 2   50ma       1047 non-null   float64
 3   20ma       1047 non-null   float64
 4   Up_Down    1047 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 49.1 KB


In [12]:
#Escalamos los valores con un MinMaxScaler
values = data.values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

values = scaled


#Dividimos los datos entre train y test
train, test = train_test_split(values, test_size=0.2)


#Separamos los inputs de los outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]


#Generamos el reshape para poder incluir los valores en el modelo LSTM
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(837, 1, 4) (837,) (210, 1, 4) (210,)


## Creamos el modelo

In [13]:
#Generamos arquitectura del modelo
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

In [37]:
#Entrenamos el modelo
history = model.fit(train_X, train_y, epochs=200, batch_size=144, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Epoch 1/200
6/6 - 0s - loss: 0.4343 - val_loss: 0.4540
Epoch 2/200
6/6 - 0s - loss: 0.4343 - val_loss: 0.4540
Epoch 3/200
6/6 - 0s - loss: 0.4342 - val_loss: 0.4540
Epoch 4/200
6/6 - 0s - loss: 0.4342 - val_loss: 0.4540
Epoch 5/200
6/6 - 0s - loss: 0.4342 - val_loss: 0.4540
Epoch 6/200
6/6 - 0s - loss: 0.4341 - val_loss: 0.4541
Epoch 7/200
6/6 - 0s - loss: 0.4341 - val_loss: 0.4541
Epoch 8/200
6/6 - 0s - loss: 0.4341 - val_loss: 0.4541
Epoch 9/200
6/6 - 0s - loss: 0.4340 - val_loss: 0.4541
Epoch 10/200
6/6 - 0s - loss: 0.4340 - val_loss: 0.4542
Epoch 11/200
6/6 - 0s - loss: 0.4340 - val_loss: 0.4542
Epoch 12/200
6/6 - 0s - loss: 0.4339 - val_loss: 0.4542
Epoch 13/200
6/6 - 0s - loss: 0.4339 - val_loss: 0.4542
Epoch 14/200
6/6 - 0s - loss: 0.4339 - val_loss: 0.4543
Epoch 15/200
6/6 - 0s - loss: 0.4338 - val_loss: 0.4543
Epoch 16/200
6/6 - 0s - loss: 0.4337 - val_loss: 0.4543
Epoch 17/200
6/6 - 0s - loss: 0.4337 - val_loss: 0.4543
Epoch 18/200
6/6 - 0s - loss: 0.4337 - val_loss: 0.4543
E

Epoch 147/200
6/6 - 0s - loss: 0.4293 - val_loss: 0.4539
Epoch 148/200
6/6 - 0s - loss: 0.4292 - val_loss: 0.4539
Epoch 149/200
6/6 - 0s - loss: 0.4293 - val_loss: 0.4539
Epoch 150/200
6/6 - 0s - loss: 0.4292 - val_loss: 0.4539
Epoch 151/200
6/6 - 0s - loss: 0.4292 - val_loss: 0.4539
Epoch 152/200
6/6 - 0s - loss: 0.4292 - val_loss: 0.4539
Epoch 153/200
6/6 - 0s - loss: 0.4291 - val_loss: 0.4539
Epoch 154/200
6/6 - 0s - loss: 0.4291 - val_loss: 0.4539
Epoch 155/200
6/6 - 0s - loss: 0.4291 - val_loss: 0.4539
Epoch 156/200
6/6 - 0s - loss: 0.4290 - val_loss: 0.4538
Epoch 157/200
6/6 - 0s - loss: 0.4290 - val_loss: 0.4538
Epoch 158/200
6/6 - 0s - loss: 0.4290 - val_loss: 0.4538
Epoch 159/200
6/6 - 0s - loss: 0.4289 - val_loss: 0.4538
Epoch 160/200
6/6 - 0s - loss: 0.4289 - val_loss: 0.4537
Epoch 161/200
6/6 - 0s - loss: 0.4289 - val_loss: 0.4537
Epoch 162/200
6/6 - 0s - loss: 0.4289 - val_loss: 0.4537
Epoch 163/200
6/6 - 0s - loss: 0.4288 - val_loss: 0.4537
Epoch 164/200
6/6 - 0s - loss: 

In [30]:
#Generamos plot con los resultados
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

AttributeError: module 'pandas' has no attribute 'plot'

In [46]:
results = pd.DataFrame(history.history)

In [48]:
results.plot()

AttributeError: module 'matplotlib' has no attribute 'rcParams'